In [ ]:
import requests
from datetime import datetime
import json
import time
import pandas as pd
from time import mktime

In [ ]:
class PolgonData(object):

    def __init__(self):
        self.params=params = (('apiKey', 'M_PKVL_rqHZI7VM9ZYO_hwPiConz5rIklx893F'),)

    def PolygonLastTrades(self,symbol):
        # Make use of Tickers
        requesturl='https://api.polygon.io/v1/last/stocks/'+symbol
        response = requests.get(requesturl, params=self.params)
        return json.loads(response.text)

    def PolygonHistoricTrades(self, date=None, symbol=None,startTS=None,endTS=None,limitresult=10):
        requesturl='https://api.polygon.io/v2/ticks/stocks/nbbo/'+symbol+'/'+date+'?timestamp='+startTS+'&timestampLimit='+'1581695521575883997'+'&limit='+limitresult
        print(requesturl)
        response = requests.get(requesturl, params=self.params)
        return json.loads(response.text)
    
    def PolygonAggregdateData(self):
        # Make use of Tickers, Date and Limit
        requesturl='https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/minute/2020-02-14/2020-02-15'
        response = requests.get(requesturl, params=self.params)
        return json.loads(response.text)

In [ ]:
from datetime import datetime

class DateTimeManipulation(object):
    
    def __init__(self,date=None):
        self.date=date

    # Returns timestamp with milliseconds
    def unix_time_millis(self,dt):
        epoch = datetime.utcfromtimestamp(0)
        tsDate=(dt - epoch).total_seconds() * 1000.0
        tsDate=str(int(tsDate))+'000000'
        return tsDate

    def convertStringDateToTS(self,starttime='9:30:00',endtime='4:00:00'):
        marketOpenTSStr = self.date +' '+ starttime
        marketCloseTSStr = self.date +' ' + endtime
        
        marketTimeStamps={}
        marketTimeStamps['marketOpenTS']=self.unix_time_millis(datetime.strptime(marketOpenTSStr,'%Y-%m-%d %H:%M:%S'))
        marketTimeStamps['marketCloseTS']=self.unix_time_millis(datetime.strptime(marketCloseTSStr,'%Y-%m-%d %H:%M:%S'))
        return marketTimeStamps
    
    def getHumanTime(self,ts,getMilliSecondsAlso=True):
        try:
            s, ms = divmod(ts, 1000000000)
            if getMilliSecondsAlso:
                return datetime(*time.gmtime(s)[:6]),ms
            else:
                return datetime(*time.gmtime(s)[:6])
            #print('{}.{:03d}'.format(time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(s)), ms))
        except AttributeError:
            print("Attribute Error Occured")
            print(ts)
            print(s)
            print(ms)
            

# Process the ticker data

XLK: Technology Select Sector SPDR Fund <br>
Inception Date: 1998-12-16 <br>
Fund Holdings as of: 2020-02-14 <br>
"Total Assets Under Management (in thousands):	28798933" <br>
Shares Outstanding: 282406000 <br>
Expense Ratio: 0.13% <br>
Tracks This Index: Technology Select Sector Index <br>
ETFdb.com Category: Technology Equities <br>
Issuer: State Street SPDR <br>
Structure: ETF <br>
ETF Home Page: https://etfdb.com/etf/XLK <br>


In [4]:
date='2020-02-14'
starttime='9:30:00'
endtime='16:00:00'

# Taking in ETF List
data=pd.read_csv("XLK-holdings20200214.csv")
symbols=list(data['Symbol'].values)+['XLK']

# Create an object of date when we need and time between which we need data
datetimeObj=DateTimeManipulation(date)
marketTimeStamps=datetimeObj.convertStringDateToTS(starttime=starttime,endtime=endtime)

tickHistData={}
for symbol in symbols:
    data=PolgonData().PolygonHistoricTrades(date=date,symbol=symbol,startTS=str(marketTimeStamps['marketOpenTS']),endTS=str(marketTimeStamps['marketCloseTS']),limitresult=str(5000))
    tickHistData[symbol]=data
    print(data['ticker'])
    print(datetimeObj.getHumanTime(data['results'][-1]['t']))

# Meaning of Each Columns

In [5]:
tickHistData['AAPL']['map']

{'t': {'name': 'sip_timestamp', 'type': 'int64'},
 'c': {'name': 'conditions', 'type': '[]int'},
 'i': {'name': 'indicators', 'type': '[]int'},
 'p': {'name': 'bid_price', 'type': 'float64'},
 'P': {'name': 'ask_price', 'type': 'float64'},
 'S': {'name': 'ask_size', 'type': 'int'},
 'y': {'name': 'participant_timestamp', 'type': 'int64'},
 'f': {'name': 'trf_timestamp', 'type': 'int64'},
 'q': {'name': 'sequence_number', 'type': 'int'},
 's': {'name': 'bid_size', 'type': 'int'},
 'x': {'name': 'bid_exchange', 'type': 'int'},
 'X': {'name': 'ask_exchange', 'type': 'int'},
 'z': {'name': 'tape', 'type': 'int'}}

In [6]:
finalDF=[]
appendData=[]
for key,value in tickHistData.items():
    df=pd.DataFrame.from_dict(value['results'])
    df['Symbol']=key
    appendData.append(df)

finalDF= pd.concat(appendData)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [7]:
# Delete conditions
del finalDF['c']
# Delete sequence number
del finalDF['q']
# Delete indicators
del finalDF['i']

In [9]:
# Clean the DataFrame
finalDF['t']=finalDF['t'].apply(lambda x:DateTimeManipulation().getHumanTime(x,getMilliSecondsAlso=False))
finalDF['y']=finalDF['y'].apply(lambda x:DateTimeManipulation().getHumanTime(x,getMilliSecondsAlso=False))
finalDF['Spread']=finalDF['P']-finalDF['p']
finalDF['MidPrice']=(finalDF['P']+finalDF['p'])/2

In [10]:
finalDF

,P,S,Symbol,X,p,s,t,x,y,z,Spread,MidPrice
0,184.24,5,MSFT,11,184.01,1,2020-02-14 09:31:34,12,2020-02-14 09:31:34,3,0.23,184.125
1,184.24,5,MSFT,11,184.02,1,2020-02-14 09:31:40,12,2020-02-14 09:31:40,3,0.22,184.130
2,184.23,1,MSFT,11,184.02,1,2020-02-14 09:32:30,12,2020-02-14 09:32:30,3,0.21,184.125
3,184.22,5,MSFT,11,184.02,1,2020-02-14 09:32:30,12,2020-02-14 09:32:30,3,0.20,184.120
4,184.24,5,MSFT,11,184.02,1,2020-02-14 09:34:27,12,2020-02-14 09:34:27,3,0.22,184.130
5,184.24,5,MSFT,11,184.01,1,2020-02-14 09:34:49,12,2020-02-14 09:34:49,3,0.23,184.125
6,184.24,5,MSFT,11,184.00,8,2020-02-14 09:34:49,11,2020-02-14 09:34:49,3,0.24,184.120
7,184.25,6,MSFT,11,184.15,3,2020-02-14 09:46:41,12,2020-02-14 09:46:41,3,0.10,184.200
8,184.24,5,MSFT,11,184.15,3,2020-02-14 09:46:53,12,2020-02-14 09:46:53,3,0.09,184.195
9,184.24,5,MSFT,11,184.00,7,2020-02-14 09:50:32,11,2020-02-14 09:50:32,3,0.24,184.120


In [11]:
dfGroupedByObject=finalDF.groupby([finalDF['t'].dt.hour,finalDF['t'].dt.minute,finalDF['Symbol']])['MidPrice'].mean()

In [27]:
dfGroupedByObject=finalDF.groupby([finalDF['t'].dt.hour,finalDF['Symbol']])['MidPrice'].mean()

In [28]:
dfGroupedByObject

t   Symbol
9   AAPL      326.096727
    AMD        55.198632
    ANET      215.350000
    CSCO       47.498971
    IBM       154.660000
    INTC       67.745000
    MA        336.445000
    MSFT      184.139000
    MU         60.152222
    NVDA      288.350000
    ORCL       55.385111
    PYPL      120.763750
    QCOM       90.421356
    V         207.770140
    XLK       101.889509
10  AAPL      326.186623
    AMAT       67.916667
    AMD        55.234758
    ANET      223.300000
    CSCO       47.500000
    IBM       154.555040
    INTC       67.814231
    MA        336.471829
    MSFT      184.397594
    MU         60.169630
    MXIM       31.770000
    NOW       325.624167
    NVDA      289.900867
    ORCL       55.438440
    QCOM       90.374048
                 ...    
15  DXC        30.153244
    FFIV      127.706321
    FISV      121.005797
    FLIR       58.612052
    FLT       317.124280
    FTNT      118.705432
    INTU      300.766475
    IPGP      145.603825
    IT        